In [1]:
!nvidia-smi

Tue Feb 15 09:56:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8     3W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision, AUC, binary_accuracy, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from helper_functions import mcc_metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from transformers import AutoTokenizer

In [3]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [4]:
data_new = data.dropna(subset=['IUPAC_name'])
X = data_new['IUPAC_name']
target=data_new['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('gumgo91/IUPAC_BERT', from_pt = True)

In [6]:
iupac_data = tokenizer(list(X),  truncation=True, padding=True, max_length=256)
iupac_data = iupac_data['input_ids']

In [7]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(iupac_data, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [8]:
len(y_classification)

6170

In [9]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([3992, 2178], dtype=int64))

In [10]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [11]:
word_in_vocab = tokenizer.vocab
len(word_in_vocab)

1500

In [12]:
np.array(xtrain[0]).shape

(256,)

In [13]:
input = Input(shape = (256,))

x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=256, 
                        input_length=None,
                        )(input)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(64, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [14]:
class_weights

array([0.77582335, 1.4063772 ])

In [15]:
class_weights = {0: 0.77582335,
                1: 1.4063772}
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'iupac'+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(33), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives()])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)

Epoch 1/60
130/130 [==============================] - ETA: 0s - loss: 0.5571 - binary_accuracy: 0.7559 - auc: 0.7900 - mcc_metric: nan - precision: 0.6709 - true_negatives: 2227.0000 - true_positives: 907.0000 - false_negatives: 567.0000 - false_positives: 445.0000
Epoch 00001: val_auc improved from -inf to 0.86767, saving model to F:/bbb/model/nlp\model_33iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


130/130 [==============================] - 57s 366ms/step - loss: 0.5571 - binary_accuracy: 0.7559 - auc: 0.7900 - mcc_metric: nan - precision: 0.6709 - true_negatives: 2227.0000 - true_positives: 907.0000 - false_negatives: 567.0000 - false_positives: 445.0000 - val_loss: 0.4891 - val_binary_accuracy: 0.7658 - val_auc: 0.8677 - val_mcc_metric: 0.5292 - val_precision: 0.6196 - val_true_negatives: 390.0000 - val_true_positives: 215.0000 - val_false_negatives: 53.0000 - val_false_positives: 132.0000
Epoch 2/60
130/130 [==============================] - ETA: 0s - loss: 0.4133 - binary_accuracy: 0.8222 - auc: 0.8921 - mcc_metric: 0.6319 - precision: 0.7211 - true_negatives: 2207.0000 - true_positives: 1202.0000 - false_negatives: 272.0000 - false_positives: 465.0000
Epoch 00002: val_auc improved from 0.86767 to 0.89508, saving model to F:/bbb/model/nlp\model_33iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


130/130 [==============================] - 45s 350ms/step - loss: 0.4133 - binary_accuracy: 0.8222 - auc: 0.8921 - mcc_metric: 0.6319 - precision: 0.7211 - true_negatives: 2207.0000 - true_positives: 1202.0000 - false_negatives: 272.0000 - false_positives: 465.0000 - val_loss: 0.3844 - val_binary_accuracy: 0.8190 - val_auc: 0.8951 - val_mcc_metric: 0.6087 - val_precision: 0.7193 - val_true_negatives: 442.0000 - val_true_positives: 205.0000 - val_false_negatives: 63.0000 - val_false_positives: 80.0000
Epoch 3/60
130/130 [==============================] - ETA: 0s - loss: 0.3462 - binary_accuracy: 0.8420 - auc: 0.9254 - mcc_metric: 0.6772 - precision: 0.7304 - true_negatives: 2193.0000 - true_positives: 1298.0000 - false_negatives: 176.0000 - false_positives: 479.0000
Epoch 00003: val_auc improved from 0.89508 to 0.89968, saving model to F:/bbb/model/nlp\model_33iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33iupac.tf\assets


130/130 [==============================] - 51s 392ms/step - loss: 0.3462 - binary_accuracy: 0.8420 - auc: 0.9254 - mcc_metric: 0.6772 - precision: 0.7304 - true_negatives: 2193.0000 - true_positives: 1298.0000 - false_negatives: 176.0000 - false_positives: 479.0000 - val_loss: 0.3851 - val_binary_accuracy: 0.8114 - val_auc: 0.8997 - val_mcc_metric: 0.6002 - val_precision: 0.6901 - val_true_negatives: 425.0000 - val_true_positives: 216.0000 - val_false_negatives: 52.0000 - val_false_positives: 97.0000
Epoch 4/60
130/130 [==============================] - ETA: 0s - loss: 0.3074 - binary_accuracy: 0.8755 - auc: 0.9407 - mcc_metric: 0.7457 - precision: 0.7821 - true_negatives: 2302.0000 - true_positives: 1328.0000 - false_negatives: 146.0000 - false_positives: 370.0000
Epoch 00004: val_auc did not improve from 0.89968
130/130 [==============================] - 19s 145ms/step - loss: 0.3074 - binary_accuracy: 0.8755 - auc: 0.9407 - mcc_metric: 0.7457 - precision: 0.7821 - true_negatives: 23

In [16]:
model.load_weights(save(33))
model.evaluate(np.array(xtest), np.array(ytest))

39/39 [==============================] - 2s 42ms/step - loss: 0.3869 - binary_accuracy: 0.8120 - auc: 0.9019 - mcc_metric: 0.6097 - precision: 0.7000 - true_negatives: 645.0000 - true_positives: 357.0000 - false_negatives: 79.0000 - false_positives: 153.0000


[0.38693147897720337,
 0.8119935393333435,
 0.9018661379814148,
 0.6096577048301697,
 0.699999988079071,
 645.0,
 357.0,
 79.0,
 153.0]

In [17]:
model.evaluate(np.array(xvalid), np.array(yvalid))

25/25 [==============================] - 1s 43ms/step - loss: 0.3851 - binary_accuracy: 0.8114 - auc: 0.8997 - mcc_metric: 0.6002 - precision: 0.6901 - true_negatives: 425.0000 - true_positives: 216.0000 - false_negatives: 52.0000 - false_positives: 97.0000


[0.3851379454135895,
 0.8113924264907837,
 0.8996754884719849,
 0.600176215171814,
 0.690095841884613,
 425.0,
 216.0,
 52.0,
 97.0]